In [2]:
1!pip install lightgbm

# EXTRA MILLE

In [17]:
!pip install catboost

    100% |████████████████████████████████| 60.6MB 342kB/s 


In [20]:

import pandas as pd
from sklearn import preprocessing
import numpy as np
import gc
import lightgbm as lgb

from scipy import sparse
from scipy.sparse.linalg import svds

from sklearn.metrics import roc_auc_score

from scipy.sparse import csr_matrix

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from catboost import CatBoostClassifier


In [21]:
train=pd.read_csv("data_train.csv",sep=",",encoding="ISO-8859-1")
test=pd.read_csv("data_test.csv",sep=",",encoding="ISO-8859-1")

In [22]:
train["TIPO_BECA"].value_counts()

SIN BECA                                154995
CONVENIO POLICIA NACIONAL                37308
DSCTO. PADRES CON VARIOS HIJOS           16246
BECA RENDIMIENTO Y STCION. ECONOMICA     12194
CONVENIO EJERCITO PERUANO                 9228
CONVENIO FUERZA AEREA DEL PERU            7529
DSCTO. HIJOS DE GRADUADOS DE URP          7239
CONVENIO MARINA DE GUERRA DEL PERU        5475
HIJO DE TRABAJADOR ADMINISTRATIVO         2487
DEPORTITAS CALIFICADOS                    1039
HIJO DE DOCENTE TIEMPO PARCIAL             955
BECA ORFANDAD                              876
HIJO DE DOCENTE TIEMPO COMPLETO            384
BECA PRIMEROS PUESTOS REGULAR              129
TRABAJADOR ADMINISTRATIVO                  110
BECA_HIJO TRAB_JUBILADO                      7
CONVENIO HOSP.DA.CARRION                     7
Name: TIPO_BECA, dtype: int64

In [23]:
len(train)

256215

In [24]:
len(train["COD_ALUMNO"].unique())

256215

In [25]:
test.head(4)

,COD_ALUMNO,SEMESTRE,TIPO_SEMESTRE,CARRERA,SITUACION_MATRICULA,CURSOSAPROBADOS,CURSOSDESAPROBADOS,CURSOSNSP,CREDAPROBADOS,CREDDESAPROBADOS,...,PROV_TRASLADO_EXTERNO,PERIODOS_DESERTO_ULT_SEM_URP,CARRERA_ACTUAL,PERIODOS_URP_CARRERA_ACTUAL_REGULAR,PERIODOS_URP_CARRERA_ACTUAL,PROVIENE_TRASLADO_INTERNO_CARRERA_ACTUAL,PRIMER_PERIODO_URP_CARRERA_ACTUAL,ULTIMO_PERIODO_URP_CARRERA_ACTUAL,SITUACION_EGRESO,TIPO_BECA
0,URP000021,20181,REGULAR CICLO I,Turismo;Hoteleria y Gastronomia,0,0,0,0,0.0,0.0,...,0,21,Turismo;Hoteleria y Gastronomia,1,0,0,20081.0,20081.0,NO EGRESO,SIN BECA
1,URP000022,20182,REGULAR CICLO II,Turismo;Hoteleria y Gastronomia,0,0,0,0,0.0,0.0,...,0,21,Turismo;Hoteleria y Gastronomia,1,0,0,20081.0,20081.0,NO EGRESO,SIN BECA
2,URP000058,20181,REGULAR CICLO I,Contabilidad y Finanzas,0,0,0,0,0.0,0.0,...,0,13,Contabilidad y Finanzas,6,4,0,20081.0,20121.0,NO EGRESO,SIN BECA
3,URP000059,20182,REGULAR CICLO II,Contabilidad y Finanzas,0,0,0,0,0.0,0.0,...,0,13,Contabilidad y Finanzas,6,4,0,20081.0,20121.0,NO EGRESO,SIN BECA


In [26]:
len(train.columns)

126

In [27]:
len(test.columns)

116

In [28]:
train["COD_ALUMNO"].isin(test["COD_ALUMNO"]).value_counts()

False    256215
Name: COD_ALUMNO, dtype: int64

In [29]:
train["SEM_NOTAS_HIST_ULTIMO_CARRERA"].value_counts()

20182    50554
20190    16592
20172    12878
20162    12664
20181    12432
20152    11148
20142    10885
20141     9993
20161     9303
20171     9236
20131     8944
20151     8741
20132     8170
20121     7358
20111     7052
20122     6931
20102     6640
20112     6601
20101     6493
20091     5300
20092     4384
20082     3921
20081     2304
20160     1288
20180     1275
20150     1211
20081     1140
20140     1075
20170     1051
20120     1016
         ...  
20141      455
20152      415
20091      371
20142      350
20102      312
20090      291
20092      264
20151      231
20172      229
20112      228
20182      227
20101      222
20161      215
20111      187
20162      164
20181      148
20121      114
20171      109
20190       96
20130       81
20180       80
20090       60
20150       42
20100       40
20140       36
20170       36
20110       30
20160       30
20120       20
NINGU        2
Name: SEM_NOTAS_HIST_ULTIMO_CARRERA, Length: 67, dtype: int64

In [30]:
train["SEMESTRE"].value_counts()

20172    21075
20171    20586
20162    19814
20161    19422
20152    18577
20151    18065
20142    17214
20141    16704
20132    15368
20131    14735
20122    13222
20121    12450
20112    10758
20111     9814
20102     7860
20101     6874
20092     5106
20091     4242
20082     2655
20081     1674
Name: SEMESTRE, dtype: int64

In [31]:
train["SEMESTRE_FASE"]=(train["SEMESTRE"])%10

test["SEMESTRE_FASE"]=(test["SEMESTRE"])%10

In [32]:


train.loc[train["SEM_NOTAS_HIST_INICIA_CARRERA"]=="NINGU","SEM_NOTAS_HIST_INICIA_CARRERA"]=np.nan

train.loc[train["SEM_NOTAS_HIST_ULTIMO_CARRERA"]=="NINGU","SEM_NOTAS_HIST_ULTIMO_CARRERA"]=np.nan

for x in train,test:

    x.loc[x["SEM_NOTAS_HIST_INICIA_CARRERA"].notnull(),"SEM_NOTAS_HIST_INICIA_CARRERA_int"]=    x.loc[x["SEM_NOTAS_HIST_INICIA_CARRERA"].notnull(),"SEM_NOTAS_HIST_INICIA_CARRERA"].astype(int)
    
    x.loc[x["SEM_NOTAS_HIST_ULTIMO_CARRERA"].notnull(),"SEM_NOTAS_HIST_ULTIMO_CARRERA_int"]=    x.loc[x["SEM_NOTAS_HIST_ULTIMO_CARRERA"].notnull(),"SEM_NOTAS_HIST_ULTIMO_CARRERA"].astype(int)
    
    

In [33]:
for x in train,test:

    x["dur"]=(x["SEM_NOTAS_HIST_ULTIMO_CARRERA_int"]//10-x["SEM_NOTAS_HIST_INICIA_CARRERA_int"]//10)*12+(x["SEM_NOTAS_HIST_ULTIMO_CARRERA_int"]%10-x["SEM_NOTAS_HIST_INICIA_CARRERA_int"]%10)

In [34]:
for x in ["SEM_NOTAS_HIST_ULTIMO_CARRERA_int","SEM_NOTAS_HIST_INICIA_CARRERA_int"]:
    
    train[x+"_fase"]=train[x]%10
    test[x+"_fase"]=test[x]%10

In [35]:
for x in train,test:

    x["dur"]=x.loc[x["dur"].notnull(),"dur"].astype(int)

In [36]:
s=[x for x in train.columns if "BOLETA" in x]

In [37]:
train[s].isnull().sum()

SITUACION_DEUDA_BOLETA_1     0
SITUACION_DEUDA_BOLETA_2     0
SITUACION_DEUDA_BOLETA_3     0
SITUACION_DEUDA_BOLETA_4     0
SITUACION_DEUDA_BOLETA_5     0
SITUACION_DEUDA_BOLETA_6     0
SITUACION_DEUDA_BOLETA_7     0
SITUACION_DEUDA_BOLETA_8     0
SITUACION_DEUDA_BOLETA_9     0
SITUACION_DEUDA_BOLETA_10    0
dtype: int64

In [38]:
train["PERIODOS_URP_REGULAR"].describe()

count    256213.000000
mean          8.260248
std           5.070450
min           0.000000
25%           3.000000
50%           9.000000
75%          12.000000
max          22.000000
Name: PERIODOS_URP_REGULAR, dtype: float64

In [39]:
test["PERIODOS_URP_REGULAR"].describe()

count    44072.000000
mean         6.024460
std          4.486279
min          0.000000
25%          2.000000
50%          5.000000
75%          9.000000
max         22.000000
Name: PERIODOS_URP_REGULAR, dtype: float64

In [40]:
tr=[x for x in s if "BOLETA_1" not in x]

train["target"]=((train[s].sum(axis=1))>0)*1

In [41]:
train["target"].value_counts()/len(train)

0    0.636657
1    0.363343
Name: target, dtype: float64

In [42]:
s=[x for x in train.columns if "BOLETA" in x]


categorical=train.select_dtypes(include=[object]).columns

categorical=[x for x in categorical if x not in s and "COD_ALUMNO" not in x ]

In [43]:
categorical

['TIPO_SEMESTRE',
 'CARRERA',
 'PORC_ASISTENCIA',
 'VIVIENDA_DONDE_VIVE',
 'TIPO_VIVIENDA_DONDE_VIVE',
 'VIVE_CON',
 'SEXO',
 'CDEPA_DESCRIPCION',
 'CPROV_DESCRIPCION',
 'CDIST_DESCRIPCION',
 'ESTADO_CIVIL',
 'TIPO_COLEGIO',
 'SEM_NOTAS_HIST_INICIA_CARRERA',
 'SEM_NOTAS_HIST_ULTIMO_CARRERA',
 'CARRERA_ACTUAL',
 'SITUACION_EGRESO',
 'TIPO_BECA']

In [44]:
for x in train,test:

    x.loc[x["PORC_ASISTENCIA"]=="SIN ASISTENCIA TOMADA","PORC_ASISTENCIA"]=np.nan

    x.loc[x["PORC_ASISTENCIA"]!="SIN ASISTENCIA TOMADA","PORC_ASISTENCIA"]=x.loc[x["PORC_ASISTENCIA"]!="SIN ASISTENCIA TOMADA","PORC_ASISTENCIA"].astype(float)



In [45]:
test["PORC_ASISTENCIA"].describe()

count    22575.000000
mean        74.681980
std         18.988409
min          0.000000
25%         66.094326
50%         78.873239
75%         88.311688
max        100.000000
Name: PORC_ASISTENCIA, dtype: float64

In [48]:
#for x in [
#"CARRERA",
#"VIVIENDA_DONDE_VIVE",
#"TIPO_VIVIENDA_DONDE_VIVE",
#"VIVE_CON",
#"CDEPA_DESCRIPCION",
#"CPROV_DESCRIPCION",
#"CDIST_DESCRIPCION",
#"TIPO_COLEGIO",
#"SEM_NOTAS_HIST_INICIA_CARRERA",
#"SEM_NOTAS_HIST_ULTIMO_CARRERA",
#"CARRERA_ACTUAL",
#"SITUACION_EGRESO",
#"TIPO_BECA"]:



#train, test=tam(train,test,["SEMESTRE"])
    


In [49]:
no_usar=s+["target"]+["COD_ALUMNO"]



features=[x for x in train.columns if x not in no_usar]

cat_ind=[features.index(x) for x in categorical]

In [50]:
for l in categorical:
    
    print(l)
    
    
    me = preprocessing.LabelEncoder()
    
    train.loc[train[l].notnull(),l]=train.loc[train[l].notnull(),l].astype(str)
    
    test.loc[test[l].notnull(),l]=test.loc[test[l].notnull(),l].astype(str)
    
    train.loc[train[l].isnull(),l]="na"
    test.loc[test[l].isnull(),l]="na"

    
    me.fit(list(train[l].dropna())+list(test[l].dropna()))
    
    

    train.loc[~train[l].isnull(),l]=me.transform(train.loc[~train[l].isnull(),l])

    test.loc[~test[l].isnull(),l]=me.transform(test.loc[~test[l].isnull(),l])
    


TIPO_SEMESTRE
CARRERA
PORC_ASISTENCIA
VIVIENDA_DONDE_VIVE
TIPO_VIVIENDA_DONDE_VIVE
VIVE_CON
SEXO
CDEPA_DESCRIPCION
CPROV_DESCRIPCION
CDIST_DESCRIPCION
ESTADO_CIVIL
TIPO_COLEGIO
SEM_NOTAS_HIST_INICIA_CARRERA
SEM_NOTAS_HIST_ULTIMO_CARRERA
CARRERA_ACTUAL
SITUACION_EGRESO
TIPO_BECA


In [ ]:
kf_previo=StratifiedKFold(n_splits=5,random_state=256,shuffle=True)

i=1

r=[]

importancias=[]

for train_index,test_index in kf_previo.split(train,train["target"]):

    model=CatBoostClassifier(iterations=30000, depth=4, learning_rate=0.1,l2_leaf_reg=8, rsm=0.95,bootstrap_type="Bernoulli",subsample=0.9,eval_metric='AUC',use_best_model=True,early_stopping_rounds=50,verbose=25,cat_features=cat_ind)

    model.fit(train.loc[train_index,features], train.loc[train_index,"target"], eval_set=(train.loc[test_index,features] ,train.loc[test_index,"target"]),plot=False)
    
    
    print ("Fold_"+str(i))
    a= (roc_auc_score(train.loc[test_index,"target"],model.predict_proba(train.loc[test_index,features].values)[:,1]))
    
    train.loc[test_index,"probabilidad"]=model.predict_proba(train.loc[test_index,features].values)[:,1]
    
    test["TARGET_FOLD"+str(i)]=model.predict_proba(test[features].values)[:,1]

    
    r.append(a)
    print (a)
    print ("")
    
    i=i+1

print ("mean: "+str(np.mean(np.array(r))))
print ("std: "+str(np.std(np.array(r))))

0:	test: 0.8099841	best: 0.8099841 (0)	total: 615ms	remaining: 5h 7m 25s
25:	test: 0.8542614	best: 0.8542614 (25)	total: 14.4s	remaining: 4h 36m 57s
50:	test: 0.8609902	best: 0.8609902 (50)	total: 27.9s	remaining: 4h 33m 33s
75:	test: 0.8722742	best: 0.8723138 (74)	total: 41.4s	remaining: 4h 31m 41s
100:	test: 0.8782456	best: 0.8782456 (100)	total: 56.8s	remaining: 4h 40m 27s
125:	test: 0.8806402	best: 0.8806402 (125)	total: 1m 9s	remaining: 4h 36m 31s
150:	test: 0.8824308	best: 0.8824308 (150)	total: 1m 23s	remaining: 4h 35m 59s
175:	test: 0.8885347	best: 0.8885347 (175)	total: 1m 40s	remaining: 4h 42m 26s
200:	test: 0.8894423	best: 0.8894463 (199)	total: 1m 53s	remaining: 4h 39m 53s
225:	test: 0.8915666	best: 0.8915729 (224)	total: 2m 6s	remaining: 4h 38m 26s
250:	test: 0.8950144	best: 0.8950144 (250)	total: 2m 19s	remaining: 4h 35m 32s
275:	test: 0.8955252	best: 0.8955252 (275)	total: 2m 32s	remaining: 4h 32m 54s
300:	test: 0.8964558	best: 0.8964558 (300)	total: 2m 47s	remaining: 4h

In [ ]:
train[s].sum(axis=1).describe()

In [ ]:
train["PROMEDIOSINNSP"].describe()

In [ ]:
test["PROMEDIOSINNSP"].describe()

In [ ]:
train["EDAD_PROM_DOCENTE"].describe()

In [ ]:
test["EDAD_PROM_DOCENTE"].describe()

In [ ]:
train["SITUACION_MATRICULA"].value_counts()

In [ ]:
test["SITUACION_MATRICULA"].value_counts()

In [ ]:
len(test["CDIST_DESCRIPCION"].unique())

In [ ]:
for l in range(5,1000,1):
    
    l=l/1000
    
    print("quantile "+str(l))
    
    
    
    l=np.quantile(train["probabilidad"],l)
    
    print("probability: "+str(l))
    a=precision_score(train["target"],(train["probabilidad"]>l)*1)
    b=recall_score(train["target"],(train["probabilidad"]>l)*1)

    
    print(a)
    print(b)
    print(2*a*b/(a+b))

    
    
    print("")

In [ ]:
r=[x for x in test.columns if "FOLD" in x]

test["probability"]=test[r].mean(axis=1)

In [ ]:
test["Morosidad"]=(test["probability"]>0.6118935765956717)*1

In [ ]:
test[["COD_ALUMNO","Morosidad"]].to_csv("submission_16.csv",index=False)